In [213]:
import textract

In [215]:
import pandas as pd
import numpy as np
import re

In [216]:
#extraigo texto
archivo= 'tresalia\Edo Cta\cuenta_cheques_banamex PDF.pdf'

In [217]:
text = textract.process(archivo)

In [218]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [219]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [220]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,cffibanamex@
1,ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020
2,CLIENTE:
3,9999999
4,Registro F ederal de Contribuyentes:


In [221]:
#cliente
cliente_lista=[]
for i in range(len(pdf)):
    if pdf[i].lower().startswith('cliente'):
        cliente_lista.append(pdf[i])
        cliente_lista.append(pdf[i+1])
        
cliente_lista

['CLIENTE:', '9999999', 'CLIENTE:', '9999999', 'CLIENTE:', '9999999']

In [222]:
cliente= 'cliente '+cliente_lista[1]
cliente

'cliente 9999999'

In [223]:
#nombre del cliente
nombre_lista=[]
for i in pdf:
    if i.startswith("ooo"):
        nombre_lista.append(i)
        
nombre_lista

['ooo o "\' NOMBRE APELLIDO P APELLIDO M']

In [224]:
nombre=nombre_lista[0].split(' ')

In [225]:
nombre_cliente=[]
for i in nombre:
    if i.isupper():
        nombre_cliente.append(i)

In [226]:
nombre=' '.join(nombre_cliente)
nombre

'NOMBRE APELLIDO P APELLIDO M'

In [227]:
#periodo
periodo_lista=[]
for i in pdf:
    if i.startswith("RESUMEN DEL"):
        periodo_lista.append(i)
        
periodo_lista

['RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020']

In [228]:
periodo= periodo_lista[0]
periodo

'RESUMEN DEL: 01/SEP/2020 AL 30/SEP/2020'

In [229]:
#rfc
rfc_lista=[]
for i in range(len(pdf)):
    if re.search('contribuyentes', pdf[i].lower()):
        rfc_lista.append(pdf[i])
        rfc_lista.append(pdf[i+1])
        
rfc_lista

['Registro F ederal de Contribuyentes:', 'XXXX999999XX9']

In [230]:
rfc='RFC '+ rfc_lista[1]
rfc

'RFC XXXX999999XX9'

In [231]:
#localizo la tabla a partir de la palabra "detalle operaciones"
indices_detalle=[]
for i in range(len(pdf)):
    if re.search('detalle de operaciones',pdf[i].lower()):
        indices_detalle.append(i)

In [232]:
indices_detalle

[46, 77]

In [233]:
# en la automatización se puede hacer un for mas grande con los indeces para casos de muchas páginas
for i in range(len(indices_detalle)):
    print(i)

0
1


# PAGINA 1

In [20]:
tabla_pag_1= pdf[indices_detalle[0]: indices_detalle[1]]
tabla_pag_1

['DETALLE DE OPERACIONES',
 'DEPOSITOS $9,984.00',
 'FECHA 01 SEP',
 'CONCEPTO SALDO ANTERIOR PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 '01 SEP 02 SEP 10 SEP 17 SEP 17 SEP',
 'COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR',
 'RETIROS',
 'DEPOSITOS',
 'SALDO',
 '1',
 '9,021.42',
 '1',
 '1',
 '1-',
 ',_',
 ',-',
 '5,000.00',
 ',_ 3,000.00',
 '9,021.42',
 ',- 3,000.00',
 ',- 9,021.42',
 ', 3,000.00 3,000.00',
 ',_ 9,021.42',
 '000184.B08CHDA001.OD.0930.01',
 '\x0ccffibanamex@ ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 '9999999',
 'Página:',
 '2 de7',
 'NOMBRE APELLIDO P APELLIDO M',
 '7']

In [54]:
#fechas
fecha_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('fecha',tabla_pag_1[i].lower()):
        fecha_lista_pag_1.append(tabla_pag_1[i])
    elif re.match(r'\d{2} \w{3}',tabla_pag_1[i]):
        fecha_lista_pag_1.append(tabla_pag_1[i])

        
fecha_lista_pag_1

['FECHA 01 SEP', '01 SEP 02 SEP 10 SEP 17 SEP 17 SEP']

In [58]:
#separo
fechas_lista_2_pag_1=[]

for i in fecha_lista_pag_1:
    fechas_lista_2_pag_1.append(re.findall(r'\d{2} \w{3}',i))

In [61]:
fechas_lista_2_pag_1

[['01 SEP'], ['01 SEP', '02 SEP', '10 SEP', '17 SEP', '17 SEP']]

In [62]:
#aplano lista de listas
fechas_pag_1 = []
for i in fechas_lista_2_pag_1:
    for j in i:
        fechas_pag_1.append(j)

In [63]:
fechas_pag_1

['01 SEP', '01 SEP', '02 SEP', '10 SEP', '17 SEP', '17 SEP']

In [69]:
#conceptos
conceptos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('concepto',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
    elif re.match('cobro',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
    elif re.match('pago',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i])
        
conceptos_lista_pag_1

['CONCEPTO SALDO ANTERIOR PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR']

In [76]:
conceptos_p1=[]

for i in range(len(conceptos_lista_pag_1)):
    j=re.split('SALDO | COB| PA',conceptos_lista_pag_1[i])
    conceptos_p1.append(j)
conceptos_p1

[['CONCEPTO ',
  'ANTERIOR',
  'GO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000'],
 ['COBRO DE CHEQUE NUMERO 9999',
  'RO DE CHEQUE NUMERO 9999',
  'RO DE CHEQUE NUMERO 9999',
  'RO DE CHEQUE NUMERO 9999',
  'GO RECIBIDO DE BMONEX POR']]

In [84]:
#aplano lista de listas
conceptos_p2 = []
for i in conceptos_p1:
    for j in i:
        conceptos_p2.append(j)

In [85]:
conceptos_p2

['CONCEPTO ',
 'ANTERIOR',
 'GO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999',
 'RO DE CHEQUE NUMERO 9999',
 'GO RECIBIDO DE BMONEX POR']

In [87]:
conceptos_pag_1=[]
for i in range(len(conceptos_p2)):
    if conceptos_p2[i].startswith('ANTERIOR'):
        j='SALDO '+conceptos_p2[i]
        conceptos_pag_1.append(j)
    elif conceptos_p2[i].startswith('GO'):
        j='PA'+conceptos_p2[i]
        conceptos_pag_1.append(j)
    elif conceptos_p2[i].startswith('RO'):
        j='COB'+ conceptos_p2[i]
        conceptos_pag_1.append(j)
    elif conceptos_p2[i].startswith('COBRO'):
        j=conceptos_p2[i]
        conceptos_pag_1.append(j)
conceptos_pag_1

['SALDO ANTERIOR',
 'PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999',
 'COBRO DE CHEQUE NUMERO 9999',
 'PAGO RECIBIDO DE BMONEX POR']

In [88]:
#retiros
retiros_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('retiros',tabla_pag_1[i].lower()):
        retiros_lista_pag_1.append(tabla_pag_1[i])
        retiros_lista_pag_1.append(tabla_pag_1[i+1])
        
retiros_lista_pag_1

['RETIROS', 'DEPOSITOS']

In [89]:
tabla_pag_1

['DETALLE DE OPERACIONES',
 'DEPOSITOS $9,984.00',
 'FECHA 01 SEP',
 'CONCEPTO SALDO ANTERIOR PAGO RECIBIDO DE BMONEX POR ORDEN DE NOMBRE XXXXXXXXXXX CTA.ORDENANTE 000000000000000000 REF.0000000 TRASPASO A CUENTA PROPIA RASTREO: 00000000',
 '01 SEP 02 SEP 10 SEP 17 SEP 17 SEP',
 'COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 COBRO DE CHEQUE NUMERO 9999 PAGO RECIBIDO DE BMONEX POR',
 'RETIROS',
 'DEPOSITOS',
 'SALDO',
 '1',
 '9,021.42',
 '1',
 '1',
 '1-',
 ',_',
 ',-',
 '5,000.00',
 ',_ 3,000.00',
 '9,021.42',
 ',- 3,000.00',
 ',- 9,021.42',
 ', 3,000.00 3,000.00',
 ',_ 9,021.42',
 '000184.B08CHDA001.OD.0930.01',
 '\x0ccffibanamex@ ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 '9999999',
 'Página:',
 '2 de7',
 'NOMBRE APELLIDO P APELLIDO M',
 '7']

# USando libreria tabula

In [93]:
#pip install tabula-py

  Using cached tabula_py-2.2.0-py3-none-any.whl (11.7 MB)
  Using cached distro-1.5.0-py2.py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\gotsa\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [94]:
import tabula

In [106]:
tabu=tabula.read_pdf(archivo, pages='all')

In [111]:
pagina_1=tabu[1]
pagina_2=tabu[2]

In [135]:
pagina1_2=pagina_1.fillna(0).drop(['1','Unnamed: 0'], axis=1)

In [131]:
pagina1_3=pagina1_2[(pagina1_2['RETIROS']!= ',-') & (pagina1_2['RETIROS']!= ',_') & (pagina1_2['DEPOSITOS']!='-')]
pagina1_3

,FECHA,CONCEPTO,RETIROS,DEPOSITOS,SALDO
0,0,SALDO ANTERIOR,0,0,"9,021.42"
1,01 SEP,PAGO RECIBIDO DE BMONEX POR,0,0,0
2,0,ORDEN DE NOMBRE,0,0,0
3,0,XXXXXXXXXXX CT A.ORDENANTE,0,0,0
4,0,000000000000000000 REF.0000000,0,0,0
5,0,TRASPASO A CUENTA PROPIA,0,0,0
6,0,RASTREO: 00000000,0,0,0
9,0,0,0,"5,000.00",0
11,01 SEP,COBRO DE CHEQUE NUMERO 9999,"3,000.00",0,"9,021.42"
13,02 SEP,COBRO DE CHEQUE NUMERO 9999,"3,000.00",0,"9,021.42"


In [204]:
pagina2_2=pagina_2.fillna(0).drop(['1'], axis=1)

In [205]:
pagina2_3=pagina2_2[(pagina2_2['CONCEPTO ORDEN RETIROS']!= ',-') & 
                    (pagina2_2['CONCEPTO ORDEN RETIROS']!= ',_') & 
                    (pagina2_2['CONCEPTO ORDEN RETIROS']!= '_,_') &
                    (pagina2_2['CONCEPTO ORDEN RETIROS']!= '-,-') &
                    (pagina2_2['DEPOSITOS']!='-')]
pagina2_3

,FECHA,CONCEPTO ORDEN RETIROS,DEPOSITOS,SALDO
0,0,DE XXXXX XXXXXXXXXXXXXX,0,0
1,0,XXXXXXX,0,0
2,0,CTA.O RDENANTE,0,0
3,0,1000000000000000000 REF.0000000,0,0
4,0,TRASPASO A CUENTA PROPIA,0,0
6,0,RASTREO: 00000000,0,0
8,0,0,"9,000.00","8,021.42"
10,18 SEP,"COBRO DE CHEQUE NUMERO 9999 , 9,000.00",0,"9,021.42"
11,24 SEP,PAGO RECIBIDO DE BMONEX POR,0,0
12,0,ORDEN DE XXXXXXX,0,0


In [206]:
#extraer retiros
retiros=[]
for i in pagina2_3['CONCEPTO ORDEN RETIROS']:
    #if str(i).startswith('COBRO'):
        #print(i)
    if re.search(r'\.\d{2}$', str(i)):
        retiros.append(re.findall(r'\d+\,\d+\.\d{2}$', str(i))[0])
    else:
        retiros.append(0)

In [207]:
pagina2_3['RETIROS']=retiros

c:\users\gotsa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [208]:
pagina2_3.rename(columns={'CONCEPTO ORDEN RETIROS':'CONCEPTO'}, inplace=True)

In [210]:
pagina2_4= pagina2_3[['FECHA','CONCEPTO', 'RETIROS','DEPOSITOS', 'SALDO']]

In [211]:
estado= pd.concat( [pagina1_3, pagina2_4], ignore_index=True)

In [212]:
estado #tabla se puede exportar

,FECHA,CONCEPTO,RETIROS,DEPOSITOS,SALDO
0,0,SALDO ANTERIOR,0,0,"9,021.42"
1,01 SEP,PAGO RECIBIDO DE BMONEX POR,0,0,0
2,0,ORDEN DE NOMBRE,0,0,0
3,0,XXXXXXXXXXX CT A.ORDENANTE,0,0,0
4,0,000000000000000000 REF.0000000,0,0,0
5,0,TRASPASO A CUENTA PROPIA,0,0,0
6,0,RASTREO: 00000000,0,0,0
7,0,0,0,"5,000.00",0
8,01 SEP,COBRO DE CHEQUE NUMERO 9999,"3,000.00",0,"9,021.42"
9,02 SEP,COBRO DE CHEQUE NUMERO 9999,"3,000.00",0,"9,021.42"
